# Telco Customer Churn
### Focused customer retention programs
#### Context
This IBM Sample Dataset has information about Telco customers and if they left the company within the last month (churn). Each row represents a unique costumer, while the columns contains information about customer’s services, account and demographic data. 

"Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs." 

Churn is a one of the biggest problem in the telecom industry. Research has shown that the average monthly churn rate among the top 4 wireless carriers in the US is 1.9% - 2%.

#### Content
Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

##### Basic information:

- Only 7043 rows
- There are 21 columns with 19 features
- Only 11 missing values (next item).

#### The data set includes information about:
Customers who left within the last month – the column is called Churn
Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
Demographic info about customers – gender, age range, and if they have partners and dependents

In [1]:
import os
# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
os.system("pip install boto3 mlflow")

0

In [2]:
import os
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import psycopg2
import pandas as pd
from sqlalchemy import create_engine , text
from typing import Optional

import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from azure.storage.blob import BlockBlobService
from io import StringIO

In [3]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://mlflow-minio-service.mlflow.svc.cluster.local:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'BS6PpUKnW^Bkc@$l$QAsY$p#l'
os.environ['MLFLOW_BASE_URL'] = 'http://mlflow-service.mlflow.svc.cluster.local:5000'

## connect to remote server
#mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])
#
## Launch the experiment on mlflow
#experiment_name = "birla_payment_delay"
##mlflow.set_experiment(experiment_name)
#
#exp_details = mlflow.get_experiment_by_name(experiment_name)
#exp_id = exp_details.experiment_id
#
#client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [4]:
MODEL_NAME = "telecom_customer_churn"
#STORAGEACCOUNTNAME= "datalayer"
#STORAGEACCOUNTKEY= "VOTufxxiY78RLKuwZxWmMSlXVR5gB5SOWjsHhRbQpHB8uBBSVUCvSBgDEMKXuhTOAzU6gUH/b5XESaXkwqvH4w=="    
#CONTAINERNAME= "birla"

POSTGRES_ADDRESS = '52.187.120.22' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'katonic@47' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = 'postgres'
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'teleco'
TABLE_DATA = 'customer_churn_data'
TABLE_MODEL = 'customer_churn_model'
TABLE_PRED = 'customer_churn_pred'

In [5]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD


def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()

def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()

# Loading best model

In [6]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
model_df = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL).iloc[::-1].reset_index(drop=True)
model_df.head()

,model,location,run_id,experiment_id,prod_time,top_features
0,RandomForestClassifier,s3://models/67/2af8e7d9f0f04e17a2c76dc61f3c3d8...,2af8e7d9f0f04e17a2c76dc61f3c3d8a,67,2021-07-16 03:41:09.473750,"{tenure,InternetService,OnlineSecurity,OnlineB..."


In [7]:
exp_id = model_df['experiment_id'][0]
model_name = model_df['model'][0]
run_id = model_df['run_id'][0]

In [8]:
print('experiment_id :',exp_id)
print('=='*50)
print('model name :',model_name)
print('=='*50)
print('run_id :',run_id)

experiment_id : 67
model name : RandomForestClassifier
run_id : 2af8e7d9f0f04e17a2c76dc61f3c3d8a


In [9]:
top_features = model_df['top_features'][0].split('{')[1].split('}')[0].split(',')
print('Selected Features :',top_features)

Selected Features : ['tenure', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']


In [10]:
# run_id = "cc047c20511e4f2182246f4466cafc9d"
# model_uri = "runs:/" + run_id + "/model"
model = mlflow.sklearn.load_model(model_df.loc[0:,'location'][0])

## Read the data

In [11]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
data = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_DATA)

data.head() 

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2505,0
1,3962,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1466,0
2,2564,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,157,1
3,5535,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1400,0
4,6511,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,925,1


In [12]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

## Predict the data

In [13]:
customer_id = data.pop('customerID')

inference_data = data[top_features]

In [14]:
prediction = model.predict(inference_data)
prediction_probability = model.predict_proba(inference_data)[:,1]

## Save prediction to table

In [15]:
predicted_data = pd.DataFrame()
predicted_data['Experiment_ID'] = [exp_id]*len(customer_id)
predicted_data['Run_ID'] = [run_id]*len(customer_id)
predicted_data['Model_Name'] = [model_name]*len(customer_id)
predicted_data['Customer_ID'] = customer_id
predicted_data['Churn'] = prediction
predicted_data['Churn_Probabilty']=prediction_probability 
predicted_data['Prediction_time'] =pd.to_datetime(datetime.now())
predicted_data.head()

,Experiment_ID,Run_ID,Model_Name,Customer_ID,Churn,Churn_Probabilty,Prediction_time
0,67,2af8e7d9f0f04e17a2c76dc61f3c3d8a,RandomForestClassifier,5375,1,0.95,2021-07-16 04:00:12.242507
1,67,2af8e7d9f0f04e17a2c76dc61f3c3d8a,RandomForestClassifier,3962,1,0.96,2021-07-16 04:00:12.242507
2,67,2af8e7d9f0f04e17a2c76dc61f3c3d8a,RandomForestClassifier,2564,1,0.97,2021-07-16 04:00:12.242507
3,67,2af8e7d9f0f04e17a2c76dc61f3c3d8a,RandomForestClassifier,5535,1,0.94,2021-07-16 04:00:12.242507
4,67,2af8e7d9f0f04e17a2c76dc61f3c3d8a,RandomForestClassifier,6511,1,0.95,2021-07-16 04:00:12.242507


In [16]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,predicted_data,TABLE_PRED)